This notebooks will describe the steps I took to migrate Superset internal database from Sqlite3 to PostgreSQL-15 

Firstly, for safety, I'll create a copy of the sqlite3 database. The location of the database file can be found in the `superset_config.py` assigned to the variable `SQLALCHEMY_DATABASE_URI`:

```sh
cp /opt/data/superset/superset.db /opt/data/superset/superset.db.bak
```

Then, I'll [rsync](https://linux.die.net/man/1/rsync) the copied file to my machine, so I can work with it locally:

```sh
rsync -av ***:/opt/data/superset/superset.db.bak /home/bida/superset/superset.db
```


Next step is to locally build the Superset database that will be populated with the Sqlite tables. To do so, the new database URI have to replace the sqlite3 database path in the variable `SQLALCHEMY_DATABASE_URI`, with the correct user permissions beforehand. The configuration below is the credentials to the postgres container in my machine, the IP and port of the database are configured in the `.env` file:

In [1]:
from sqlalchemy import create_engine


DB_HOST = "172.27.0.2"
DB_PORT = 15432
DB_USER = "dev_admin"
DB_PASS = "admin"
DB_DATABASE = "dev_superset"
SQLALCHEMY_DATABASE_URI = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_DATABASE}"

psql_con = create_engine(SQLALCHEMY_DATABASE_URI)

### Preparing PostgreSQL database

When building the Superset container, if the postgres user and database are properly configured, the database structure will be generated with the default data in it. To ensure there won't be conflicting data, it's a good idea to clean the database before start the migration

In [2]:
import sqlite3
sqlite_db = '/home/bida/superset/superset.db'
sqlite_con = sqlite3.connect(sqlite_db)
cursor = sqlite_con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
sqlite_tables = cursor.fetchall()
sqlite_con.close()

In [3]:
from itertools import chain
sqlite_tables = list(chain(*sqlite_tables))

In [4]:
from sqlalchemy import inspect
inspector = inspect(psql_con)

psql_tables = inspector.get_table_names()

In [5]:
superset_tables = list(set(psql_tables).intersection(sqlite_tables))

#### WARNING - THIS WILL DELETE ALL DATA IN SUPERSET'S POSTGRES DATABASE 

In [6]:
with psql_con.connect() as conn:
    for table in superset_tables:
        sql = f"ALTER TABLE {table} DISABLE TRIGGER ALL; DELETE FROM {table}"
        conn.execute(sql)

/tmp/ipykernel_269513/964981679.py:4: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(sql)


### Tables - Topological Sort

Now it's time to create the tables hierarchy, respecting the foreign keys from each table. 

The rules are:
- Tables that don't contain foreign keys will have to be inserted first
- A table can't be inserted until all of its foreign key tables have been inserted  

In [7]:
fkeys = dict()
for table in superset_tables:
    fkey_dicts = inspector.get_foreign_keys(table)
    fkeys[table] = set()
    for fk in fkey_dicts:
        if table == fk['referred_table']:
            continue
        fkeys[table].add(fk['referred_table'])

In [8]:
# Result (table, {foreign_keys}):
i = 0
for fk in fkeys.items():
    if i > 5:
        break
    print(fk)
    i += 1
    

('clusters', {'ab_user'})
('ab_role', set())
('url', {'ab_user'})
('slice_email_schedules', {'slices', 'ab_user'})
('rls_filter_tables', {'tables', 'row_level_security_filters'})
('css_templates', {'ab_user'})


Now that we have calculated which tables are foreign keys to each table, we can use [graphlib](https://docs.python.org/3/library/graphlib.html) package in Python, that has a [Topological sort](https://en.wikipedia.org/wiki/Topological_sorting) algorithm. The `TopologicalSorter` will calculate that hierarchical result:

In [9]:
from graphlib import TopologicalSorter
ts = TopologicalSorter(fkeys)
tables_order = tuple(ts.static_order())

In [10]:
print(tables_order)

('ab_user', 'ab_role', 'ab_permission', 'ab_view_menu', 'keyvalue', 'ab_register_user', 'cache_keys', 'alembic_version', 'clusters', 'url', 'css_templates', 'dynamic_plugin', 'tag', 'slices', 'key_value', 'annotation_layer', 'logs', 'access_request', 'dashboards', 'dbs', 'row_level_security_filters', 'favstar', 'ab_user_role', 'ab_permission_view', 'datasources', 'tagged_object', 'slice_email_schedules', 'slice_user', 'annotation', 'embedded_dashboards', 'dashboard_user', 'user_attribute', 'dashboard_email_schedules', 'dashboard_slices', 'filter_sets', 'dashboard_roles', 'saved_query', 'report_schedule', 'query', 'alerts', 'tables', 'sl_datasets', 'sl_tables', 'rls_filter_roles', 'ab_permission_view_role', 'metrics', 'druiddatasource_user', 'columns', 'report_execution_log', 'report_recipient', 'report_schedule_user', 'tab_state', 'alert_owner', 'sql_observations', 'alert_logs', 'rls_filter_tables', 'sql_metrics', 'table_columns', 'sqlatable_user', 'table_schema')


### Populating the tables

In [11]:
import pandas as pd
from numpy.dtypes import BoolDType
from uuid import UUID

Some columns will require data parsing in its values. For instance, the boolean columns in the sqlite tables are represented with 0 and 1s (int64), while in the postgres table, they are boolean fields 

In [12]:
def parse_to_df(table: str) -> pd.DataFrame:
    sqlite_con = sqlite3.connect(sqlite_db)
    sqlite_df = pd.read_sql(f"SELECT * FROM {table}", sqlite_con)
    sqlite_con.close()
    
    sqlite_df = sqlite_df.convert_dtypes()
    psql_df = pd.read_sql_table(table, psql_con) # Empty table
    
    for column, dtype in psql_df.dtypes.items():
        if isinstance(dtype, BoolDType):
            sqlite_df[column] = sqlite_df[column].apply(lambda x: bool(x) if isinstance(x, int) else x)
        
        if column == 'uuid':
            sqlite_df[column] = sqlite_df[column].apply(lambda x: UUID(bytes=x))
        
    # https://github.com/apache/superset/pull/21284/files#diff-633d4f34919db8ab7db651657ebf8a578f437ca1f95d48c4b27c6ccd83c54640R36
    if table == 'dbs':
        sqlite_df = sqlite_df.drop(columns=['allow_multi_schema_metadata_fetch'])
    
    return sqlite_df

In [13]:
with psql_con.connect() as conn:
    for table in tables_order:
        if table == "logs":
            continue # ignore logs table
            
        df = parse_to_df(table)
        df.to_sql(table, conn, index=False, if_exists="append")
        
        print(f"{len(df)} inserted on {table}")
        
        if "id" in df.columns:
            max_id = 0 if not str(df.id.max()).isnumeric() else df.id.max()
            sql = (
                f"ALTER TABLE {table} ALTER COLUMN id SET DEFAULT nextval('{table}_id_seq'), ENABLE TRIGGER ALL; "
                f"ALTER SEQUENCE {table}_id_seq RESTART WITH {max_id + 1}; "
                
            )
            conn.execute(sql)

92 inserted on ab_user
10 inserted on ab_role
107 inserted on ab_permission
277 inserted on ab_view_menu
0 inserted on keyvalue
9 inserted on ab_register_user
492 inserted on cache_keys
1 inserted on alembic_version
0 inserted on clusters
42 inserted on url
2 inserted on css_templates
0 inserted on dynamic_plugin
6 inserted on tag
138 inserted on slices
2923 inserted on key_value
5 inserted on annotation_layer
0 inserted on access_request
65 inserted on dashboards
7 inserted on dbs
0 inserted on row_level_security_filters
27 inserted on favstar
101 inserted on ab_user_role
440 inserted on ab_permission_view
0 inserted on datasources
36 inserted on tagged_object
0 inserted on slice_email_schedules
146 inserted on slice_user
11 inserted on annotation
0 inserted on embedded_dashboards
66 inserted on dashboard_user
0 inserted on user_attribute
0 inserted on dashboard_email_schedules
87 inserted on dashboard_slices
0 inserted on filter_sets
0 inserted on dashboard_roles
18 inserted on saved

#### Checking if tables have same lengh

In [14]:
psql_len = 0
sqlite_len = 0
for table in tables_order:
    if table == "logs":
        continue
        
    psql_len += len(pd.read_sql_table(table, psql_con))
    
    sqlite_con = sqlite3.connect(sqlite_db)
    sqlite_len += len(pd.read_sql(f"SELECT * FROM {table}", sqlite_con))
    sqlite_con.close()

In [15]:
assert(psql_len == sqlite_len)